In [ ]:
import io
import os
import re
import requests
import pandas as pd
from collections import Counter
from typing import List, Set

In [ ]:
FINRAD_RAW_URL = "https://raw.githubusercontent.com/sohomghosh/FinRAD_Financial_Readability_Assessment_Dataset/main/data_sample_1500.csv"
FINRAD_TOP_K = 500
MIN_TOKEN_LENGTH = 3
MIN_REVIEW_LEN = 10
FILENAME = "binance_reviews_last5y.csv"
OUTPUT_PATH = FILENAME[:-4] + "_filtered.csv"

COMMON_FIN_WORDS = {
    "buy", "sell", "trade", "order", "limit order", "market order", "stop loss",
    "margin", "margin call", "intraday", "swing", "long", "short",
    "stock", "share", "equity", "futures", "options", "mutual fund", "sip", "ipo", "etf",
    "bond", "derivative", "portfolio", "broker", "brokerage", "commission", "fee", "tax",
    "dividend", "profit", "loss", "pnl", "realized", "unrealized", "nse", "bse", "nifty",
    "sensex", "ltp", "tick", "volume", "order id", "txn", "transaction", "txnid",
    "settlement", "upi", "bank", "demat", "dp", "scrip", "circuit breaker", "selloff",
    "panic", "panic-sell", "withdraw", "deposit", "portfolio value", "exit", "entry",
    "brokerage", "tax-loss", "tax harvesting", "brokerage fee", "kyc"
}

REGEX_PATTERNS = {
    "percent_value": re.compile(r"\b\d{1,3}(?:[.,]\d+)?\s?%"),                        # 5% / 10.5 %
    "rupee_symbol_amount": re.compile(r"₹\s?\d[\d,]*(?:\.\d+)?"),                    # ₹1,234.56
    "rs_amount": re.compile(r"\b(?:rs\.?|inr)\s?\d[\d,]*(?:\.\d+)?\b", re.I),         # rs 1200 / INR1200
    "exchange_tag": re.compile(r"\b(?:NSE|BSE|NIFTY|SENSEX)\b", re.I),
    "order_txn": re.compile(r"\b(?:order id|order#|ordernumber|txn id|transaction id|txnid)\b", re.I),
    "stop_loss_phrase": re.compile(r"\bstop[- ]?loss\b", re.I),
    "margin_call_phrase": re.compile(r"\bmargin call\b", re.I),
    "at_price_shorthand": re.compile(r"[@]\s?\d[\d,]*(?:\.\d+)?"),                    # '@ 450' or '@450'
    "percent_word": re.compile(r"\bpercent\b|\bpercentage\b", re.I),
    "price_word": re.compile(r"\bprice\b|\bclosing price\b|\bopen price\b", re.I),
}



In [ ]:
# ---------- Helper functions ----------
def download_finrad_csv(raw_url: str, timeout=30):
    try:
        headers = {"User-Agent": "python-requests/2.x"}
        r = requests.get(raw_url, headers=headers, timeout=timeout)
        r.raise_for_status()
        return r.content
    except Exception as e:
        print(f"[warning] could not download FinRAD from {raw_url}: {e}")
        return None

def load_finrad_df(raw_bytes: bytes):
    try:
        return pd.read_csv(io.BytesIO(raw_bytes), encoding="utf-8", low_memory=False)
    except Exception as e:
        print("[warning] failed to parse FinRAD CSV:", e)
        return None

# def choose_text_column(df: pd.DataFrame) -> str:
#     preferred = [c for c in df.columns if c.lower() in {"text", "sentence", "content", "sample", "excerpt"}]
#     if preferred:
#         return preferred[0]
#     # fallback: pick object dtype column with largest average length
#     obj_cols = [c for c in df.columns if df[c].dtype == "object"]
#     if not obj_cols:
#         return df.columns[0]
#     avg_lens = {c: df[c].astype(str).str.len().mean() for c in obj_cols}
#     return max(avg_lens, key=avg_lens.get)

def extract_top_tokens_from_texts(texts: List[str], top_k=500, min_len=3, stopwords: Set[str]=None):
    token_re = re.compile(r"\b[a-zA-Z]{%d,}\b" % min_len)
    counter = Counter()
    for t in texts:
        if not isinstance(t, str):
            continue
        for m in token_re.findall(t):
            token = m.lower()
            if stopwords and token in stopwords:
                continue
            counter[token] += 1
    most = [tok for tok, _ in counter.most_common(top_k)]
    return most

def compile_keyword_regex(words: List[str]) -> re.Pattern:
    escaped = [re.escape(w) for w in sorted(set(words), key=len, reverse=True) if w.strip()]
    if not escaped:
        # fallback to a safe small pattern
        return re.compile(r"\b(?:buy|sell|trade|stock|share|sip)\b", re.I)
    pattern = r"\b(?:" + "|".join(escaped) + r")\b"
    return re.compile(pattern, re.I)

def filter_reviews_by_whitelist_and_regex(reviews_df: pd.DataFrame, text_col: str, keyword_pattern: re.Pattern, regex_patterns: dict, min_len: int = 10):
    s = reviews_df[text_col].astype(str).fillna("").str.strip()
    length_ok = s.str.len() >= min_len
    is_keyword = s.str.contains(keyword_pattern, na=False)

    matched_any_regex = pd.Series(False, index=reviews_df.index)
    matched_regex_keys = [[] for _ in range(len(reviews_df))]
    for key, pat in regex_patterns.items():
        found = s.str.contains(pat, na=False)
        matched_any_regex = matched_any_regex | found
        for idx in reviews_df.index[found]:
            matched_regex_keys[idx] = matched_regex_keys[idx] + [key]

    out = reviews_df.copy()
    out["text_len"] = s.str.len()
    out["is_keyword"] = is_keyword
    out["is_regex"] = matched_any_regex
    out["matched_regexes"] = [",".join(keys) if keys else "" for keys in matched_regex_keys]
    def first_keyword_match(txt: str):
        m = keyword_pattern.search(txt)
        return m.group(0) if m else ""
    out["matched_keyword_snippet"] = s.apply(first_keyword_match)
    out["finance_flag"] = (out["is_keyword"] | out["is_regex"]) & length_ok

    def reason_row(r):
        if not r["finance_flag"]:
            return "no_match_or_too_short"
        if r["is_keyword"]:
            return f"keyword:{r['matched_keyword_snippet'] or 'match'}"
        if r["is_regex"]:
            return f"regex:{r['matched_regexes']}"
        return "matched"
    out["reason"] = out.apply(reason_row, axis=1)
    return out




In [ ]:
if not os.path.exists(FILENAME):
    raise FileNotFoundError(f"'{FILENAME}' not found in working directory. Please ensure the file is present.")
df = pd.read_csv(FILENAME, low_memory=False)
print(f"Loaded '{FILENAME}' with {len(df)} rows and columns: {list(df.columns)[:20]}")

if "content" not in df.columns:
    raise ValueError("Column 'content' not found in zerodha.csv. Please ensure your dataset has a 'content' column.")

finrad_bytes = download_finrad_csv(FINRAD_RAW_URL)
finrad_tokens = []
if finrad_bytes:
    df_fin = load_finrad_df(finrad_bytes)
    if df_fin is not None:
        text_col = "terms"
        fin_texts = df_fin[text_col].astype(str).tolist()
        stopwords = set()
        try:
            import nltk
            nltk.download('stopwords', quiet=True)
            from nltk.corpus import stopwords as sw
            stopwords = set(sw.words('english'))
        except Exception:
            stopwords = {"the", "and", "for", "with", "that", "this", "are", "was", "from", "have", "has"}
        finrad_tokens = extract_top_tokens_from_texts(fin_texts, top_k=FINRAD_TOP_K, min_len=MIN_TOKEN_LENGTH, stopwords=stopwords)
        print(f"Extracted {len(finrad_tokens)} tokens from FinRAD (top {FINRAD_TOP_K}).")
    else:
        print("Could not parse FinRAD CSV; continuing with curated common words only.")
else:
    print("FinRAD download failed; continuing with curated common words only.")

merged = set(w.lower() for w in COMMON_FIN_WORDS)
merged.update(finrad_tokens)
merged_list = sorted(merged)
print(f"Merged whitelist size: {len(merged_list)} (including curated common words and FinRAD-derived tokens)")

keyword_re = compile_keyword_regex(merged_list)

filtered_df = filter_reviews_by_whitelist_and_regex(df, text_col="content", keyword_pattern=keyword_re, regex_patterns=REGEX_PATTERNS, min_len=MIN_REVIEW_LEN)

finance_rows = filtered_df[filtered_df["finance_flag"]].copy()
finance_rows.to_csv(OUTPUT_PATH, index=False)
print(f"Filtered dataset saved to: {OUTPUT_PATH}")
print(f"Total rows: {len(df)}, Finance-related rows kept: {len(finance_rows)} ({len(finance_rows)/len(df)*100:.2f}%)")

display_cols = ["content", "finance_flag", "is_keyword", "is_regex", "matched_keyword_snippet", "matched_regexes", "reason"]
sample_display = finance_rows[display_cols].head(20)
print(sample_display.to_string(index=False))


In [30]:
print(len(filtered_df['content'].unique()))

165185


In [31]:
finance_rows.shape

(19063, 14)

In [ ]:
import re

NEGATIVE_WORDS = {
    "app", "application", "ui", "ux", "interface", "layout", "theme", "dark mode", "color", "design",
    "button", "menu", "screen", "screen freeze", "freeze", "frozen", "render", "scroll", "tap", "click",
    "slow", "lag", "lagging", "loading", "load", "responsive", "unresponsive", "crash", "crashes", "crashed",
    "bug", "bugs", "glitch", "glitches", "error", "errors", "exception",
    "login", "logout", "signin", "sign in", "signup", "sign up", "register", "registration", "password",
    "forgot password", "otp", "two-factor", "2fa", "mfa", "biometric", "fingerprint", "faceid",
    "pin", "pincode", "session expired", "session", "token", "auth", "authenticate",
    "install", "uninstall", "update", "updated", "version", "play store", "app store", "rating", "stars",
    "notification", "notifications", "permission", "permissions", "push", "push notification",
    "support", "customer support", "help", "contact", "ticket",
    "hamburger", "toolbar", "popup", "dialog", "modal"
}

In [ ]:
def compile_phrase_regex(words):
    escaped = [re.escape(w) for w in sorted(set(words), key=len, reverse=True) if w.strip()]
    if not escaped:
        return re.compile(r"$^", re.I)
    pattern = r"\b(?:" + "|".join(escaped) + r")\b"
    return re.compile(pattern, re.I)

NEGATIVE_REGEX = compile_phrase_regex(NEGATIVE_WORDS)

def negative_filter(finance_rows, text_col="content", drop=True):

    s = finance_rows[text_col].astype(str).fillna("")
    negative_matches = s.str.contains(NEGATIVE_REGEX, na=False)
    out = finance_rows.copy()
    out["negative_flag"] = negative_matches
    def _snippet(txt):
        m = NEGATIVE_REGEX.search(txt)
        return m.group(0) if m else ""
    out["negative_match_snippet"] = out[text_col].apply(_snippet)
    if drop:
        return out[~out["negative_flag"]].reset_index(drop=True)
    else:
        return out.reset_index(drop=True)


In [ ]:
final_df = negative_filter(finance_rows, text_col="content", drop=True)

final_csv_path = FILENAME[:-4] + "_filtered_no_ui.csv"
final_df.to_csv(final_csv_path, index=False)

print(f"Saved cleaned finance-only reviews (UI/login removed): {len(final_df)} rows -> {final_csv_path}")

# flagged_df = negative_filter(finance_rows, text_col="content", drop=False)
# flagged_csv_path = "zerodha_finance_filtered_flagged_ui.csv"
# flagged_df.to_csv(flagged_csv_path, index=False)
# print(f"Saved finance reviews with negative flags: {len(flagged_df)} rows -> {flagged_csv_path}")


Saved cleaned finance-only reviews (UI/login removed): 9359 rows -> binance_reviews_last5y_filtered_no_ui.csv


In [15]:
final_df.shape

(20000, 21)

In [21]:
print(len(final_df['content'].unique()))

10


In [ ]:


import io
import re
import requests
import pandas as pd
from collections import Counter
from typing import List, Set

FINRAD_RAW_URL = "https://raw.githubusercontent.com/sohomghosh/FinRAD_Financial_Readability_Assessment_Dataset/main/data_sample_1500.csv"
finrad_local_path = None  # e.g., "/path/to/data_sample_1500.csv" to use a local copy instead of downloading

TOP_K_FINRAD_TOKENS = 500   # how many top tokens to extract from FinRAD as candidate finance words
MIN_TOKEN_LENGTH = 3        # ignore very short tokens
MIN_REVIEW_LEN = 10         # minimum characters for a review to be considered
# ------------------------------

# ---------- Common curated financial words (compact set you can expand) ----------
# This is a small curated "common financial words" set to combine with FinRAD-derived tokens.
COMMON_FIN_WORDS = {
    "buy", "sell", "trade", "order", "limit order", "market order", "stop loss",
    "margin", "margin call", "intraday", "swing", "long", "short",
    "stock", "share", "equity", "futures", "options", "mutual fund", "sip", "ipo", "etf",
    "bond", "derivative", "portfolio", "broker", "brokerage", "commission", "fee", "tax",
    "dividend", "profit", "loss", "pnl", "realized", "unrealized", "nse", "bse", "nifty",
    "sensex", "ipo allotment", "ltp", "tick", "volume", "order id", "txn", "transaction",
    "settlement", "upi", "bank", "demat", "dp", "scrip", "circuit breaker", "selloff",
    "panic", "panic-sell", "withdraw", "deposit", "portfolio value", "exit", "entry",
    "brokerage", "tax-loss", "tax harvesting"
}

# ---------- regex patterns ----------
REGEX_PATTERNS = {
    "percent_value": re.compile(r"\b\d{1,3}(?:[.,]\d+)?\s?%"),                        # 5% / 10.5 %
    "rupee_symbol_amount": re.compile(r"₹\s?\d[\d,]*(?:\.\d+)?"),                    # ₹1,234.56
    "rs_amount": re.compile(r"\b(?:rs\.?|inr)\s?\d[\d,]*(?:\.\d+)?\b", re.I),         # rs 1200 / INR1200
    "exchange_tag": re.compile(r"\b(?:NSE|BSE|NIFTY|SENSEX)\b", re.I),
    "order_txn": re.compile(r"\b(?:order id|order#|ordernumber|txn id|transaction id|txnid)\b", re.I),
    "stop_loss_phrase": re.compile(r"\bstop[- ]?loss\b", re.I),
    "margin_call_phrase": re.compile(r"\bmargin call\b", re.I),
    "at_price_shorthand": re.compile(r"[@]\s?\d[\d,]*(?:\.\d+)?"),                    # '@ 450' or '@450'
    "percent_word": re.compile(r"\bpercent\b|\bpercentage\b", re.I),
    "price_word": re.compile(r"\bprice\b|\bclosing price\b|\bopen price\b", re.I),
}

# ---------- helper functions ----------
def download_finrad_csv(raw_url: str) -> bytes:
    """Download the CSV bytes from the raw GitHub URL. If it fails, raise an exception."""
    headers = {"User-Agent": "python-requests/2.x"}
    r = requests.get(raw_url, headers=headers, timeout=30)
    r.raise_for_status()
    return r.content

def load_finrad_df(raw_bytes: bytes) -> pd.DataFrame:
    """Load csv bytes into a pandas DataFrame."""
    return pd.read_csv(io.BytesIO(raw_bytes), encoding="utf-8", low_memory=False)

def choose_text_column(df: pd.DataFrame) -> str:
    """
    Heuristic to pick the most likely text column in FinRAD sample:
    prefer columns named 'text', 'sentence', 'content', otherwise choose the string column
    with largest average length.
    """
    preferred = [c for c in df.columns if c.lower() in {"text", "sentence", "content", "sample", "excerpt"}]
    if preferred:
        return preferred[0]
    # fallback: pick object dtype column with largest average length
    obj_cols = [c for c in df.columns if df[c].dtype == "object"]
    if not obj_cols:
        # as fallback, use first column
        return df.columns[0]
    avg_lens = {c: df[c].astype(str).str.len().mean() for c in obj_cols}
    # return column with max avg len
    return max(avg_lens, key=avg_lens.get)

def extract_top_tokens_from_texts(texts: List[str], top_k=500, min_len=3, stopwords: Set[str]=None) -> List[str]:
    """
    Tokenize texts, count token frequency, and return top_k tokens (lowercased).
    Very conservative tokenization: keeps alphabetic tokens.
    """
    token_re = re.compile(r"\b[a-zA-Z]{%d,}\b" % min_len)
    counter = Counter()
    for t in texts:
        if not isinstance(t, str):
            continue
        for m in token_re.findall(t):
            token = m.lower()
            if stopwords and token in stopwords:
                continue
            counter[token] += 1
    most = [tok for tok, _ in counter.most_common(top_k)]
    return most

def compile_keyword_regex(words: List[str]) -> re.Pattern:
    """
    Build a case-insensitive regex that will match any phrase in words.
    Long phrases are placed earlier to ensure greedy matching.
    """
    # Escape and sort by length to match multi-word phrases first
    escaped = [re.escape(w) for w in sorted(set(words), key=len, reverse=True) if w.strip()]
    # join using a non-capturing group; use word boundaries to reduce false positives
    pattern = r"\b(?:" + "|".join(escaped) + r")\b"
    return re.compile(pattern, re.I)

def filter_reviews_by_whitelist_and_regex(
    reviews_df: pd.DataFrame,
    text_col: str,
    keyword_pattern: re.Pattern,
    regex_patterns: dict,
    min_len: int = 10
) -> pd.DataFrame:
    s = reviews_df[text_col].astype(str).fillna("").str.strip()
    length_ok = s.str.len() >= min_len
    is_keyword = s.str.contains(keyword_pattern, na=False)

    matched_any_regex = pd.Series(False, index=reviews_df.index)
    matched_regex_keys = [[] for _ in range(len(reviews_df))]
    for key, pat in regex_patterns.items():
        found = s.str.contains(pat, na=False)
        matched_any_regex = matched_any_regex | found
        for idx in reviews_df.index[found]:
            matched_regex_keys[idx] = matched_regex_keys[idx] + [key]

    out = reviews_df.copy()
    out["text_len"] = s.str.len()
    out["is_keyword"] = is_keyword
    out["is_regex"] = matched_any_regex
    out["matched_regexes"] = [",".join(keys) if keys else "" for keys in matched_regex_keys]
    # first keyword match snippet
    def first_keyword_match(txt: str):
        m = keyword_pattern.search(txt)
        return m.group(0) if m else ""
    out["matched_keyword_snippet"] = s.apply(first_keyword_match)
    out["finance_flag"] = (out["is_keyword"] | out["is_regex"]) & length_ok

    def reason_row(r):
        if not r["finance_flag"]:
            return "no_match_or_too_short"
        if r["is_keyword"]:
            return f"keyword:{r['matched_keyword_snippet'] or 'match'}"
        if r["is_regex"]:
            return f"regex:{r['matched_regexes']}"
        return "matched"
    out["reason"] = out.apply(reason_row, axis=1)
    return out

# ---------- Main pipeline ----------
def build_and_apply_filter(reviews_df: pd.DataFrame, review_text_col: str = "review"):
    # 1) load FinRAD tokens
    finrad_texts = []
    try:
        if finrad_local_path:
            with open(finrad_local_path, "rb") as f:
                raw = f.read()
        else:
            raw = download_finrad_csv(FINRAD_RAW_URL)
        df_finrad = load_finrad_df(raw)
        text_col = "terms"
        print("FinRAD: using text column:", text_col)
        finrad_texts = df_finrad[text_col].astype(str).tolist()
    except Exception as e:
        print("Could not load FinRAD CSV automatically:", e)
        finrad_texts = []

    # 2) compute top tokens from FinRAD (if available)
    stopwords = set()
    try:
        import nltk
        nltk.download('stopwords', quiet=True)
        from nltk.corpus import stopwords as sw
        stopwords = set(sw.words('english'))
    except Exception:
        # if nltk not available, use a small default stopwords set
        stopwords = {"the", "and", "for", "with", "that", "this", "are", "was", "from", "have", "has"}

    fin_tokens = []
    if finrad_texts:
        fin_tokens = extract_top_tokens_from_texts(finrad_texts, top_k=TOP_K_FINRAD_TOKENS, min_len=MIN_TOKEN_LENGTH, stopwords=stopwords)
        print(f"Extracted {len(fin_tokens)} tokens from FinRAD (top {TOP_K_FINRAD_TOKENS})")
    else:
        print("No FinRAD texts available; skipping FinRAD-derived tokens.")

    # 3) Merge curated common list + finrad tokens
    merged = set([w.lower() for w in COMMON_FIN_WORDS])
    merged.update(fin_tokens)
    merged_list = sorted(merged)
    print("Merged whitelist size:", len(merged_list))

    # 4) compile keyword regex
    keyword_re = compile_keyword_regex(merged_list)

    # 5) apply filter to user's reviews DataFrame
    filtered = filter_reviews_by_whitelist_and_regex(reviews_df, text_col=review_text_col, keyword_pattern=keyword_re, regex_patterns=REGEX_PATTERNS, min_len=MIN_REVIEW_LEN)
    return filtered, merged_list

# ---------- Example usage ----------
if __name__ == "__main__":
    # Sample reviews DF (replace with your Zerodha reviews DataFrame)
    sample_reviews = [
        "Bought 100 shares of RELIANCE at ₹2,345.50 today",
        "App crashes when placing order - please fix",
        "Great UI, love the new color scheme!",
        "SIP failed this month and I am worried about my investments",
        "Login issue, can't access my account",
        "Sold all my holdings after panic sell, huge loss of 12%",
        "Brokerage fee seems high compared to other platforms",
        "Why is the KYC pending for 2 days?"
    ]
    reviews_df = pd.DataFrame({"review": sample_reviews})
    filtered_df, whitelist = build_and_apply_filter(reviews_df, review_text_col="review")

    print(filtered_df[["review", "finance_flag", "is_keyword", "is_regex", "matched_keyword_snippet", "matched_regexes", "reason"]])
    # optionally save the whitelist
    pd.Series(whitelist).to_csv("merged_finance_whitelist.csv", index=False, header=False)
    print("Saved merged whitelist to merged_finance_whitelist.csv")
